In [12]:
# using selenium web driver to automate FE and do web scraping.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [13]:
def getCryptoCurrMktCap(crypto_curr):
    driver = webdriver.Chrome(r'C:/Users/leongyld/AppData/Local/Continuum/anaconda3/Lib/site-packages/selenium/webdriver/common/chromedriver.exe')
    driver.get('https://coinmarketcap.com/')
    driver.maximize_window()
    # allow the website to load all its ajax/javascripts in the bg
    driver.set_page_load_timeout(20)

    assert 'Cryptocurrency Market Capitalizations | CoinMarketCap' in driver.title
    #elem = driver.find_element_by_css_selector("navbar-form quick-search-container")
    searchbox_elem = driver.find_element_by_xpath("//*[@class='form-control quick-search-box js-quick-search tt-input']")

    searchbox_elem.clear()
    searchbox_elem.send_keys(crypto_curr)
    driver.implicitly_wait(5) # Let the user actually see something!
    searchbox_elem.send_keys(Keys.RETURN)

    # allow the website to load all its ajax/javascripts in the bg
    driver.set_page_load_timeout(20)
    crypto_hist_link = driver.find_element_by_xpath("//*[@href='/currencies/"+crypto_curr.lower()+"/historical-data/']")
    crypto_hist_link.send_keys(Keys.ENTER)

    driver.implicitly_wait(5) # Let the user actually see something!

    historicaldata_elem=driver.find_element_by_id('historical-data')
    historicaldata_elem.location_once_scrolled_into_view #scroll to the particular elem

    #retrieve the YTD performance of the crytocurrency
    reportType_ddl_elem = driver.find_element_by_id("reportrange")
    reportType_ddl_elem.click()
    reportType_item = reportType_ddl_elem.find_element_by_xpath("//li[@data-range-key='Year To Date']")
    reportType_item.click()
    #scroll down to ensure that FYTD timespan is selected
    historicaldata_elem=driver.find_element_by_id('historical-data')
    historicaldata_elem.location_once_scrolled_into_view #scroll to the particular elem

    driver.implicitly_wait(5) # Let the user actually see something!

    #def to retrieve Date and Mkt Cap of crytocurrency
    crypto_Date_list = []
    crypto_MktCap_list = []

    #loop 120 days
    for i in range(1,120):
        date_elem = driver.find_elements_by_xpath('//*[@id="historical-data"]/div/div/table/tbody/tr['+str(i)+']/td[1]')
        mktcap_elem= driver.find_elements_by_xpath('//*[@id="historical-data"]/div/div/table/tbody/tr['+str(i)+']/td[7]')
        #print (date_elem.text, mktcap_elem.text)
    
        #append to date_list
        for elem in date_elem:
            crypto_Date_list.append(elem.text)
            #append to mktcap_list
        for elem in mktcap_elem:
            crypto_MktCap_list.append(elem.text)

    #create the cypto dataframe
    cypto_curr_df=pd.DataFrame({'Date':crypto_Date_list,
                                'Market_Cap':crypto_MktCap_list
                               })

    print(cypto_curr_df.head(10))

    #capture the screenshot as test evidence!
    driver.save_screenshot(r'C:/Users/leongyld/Desktop/NUS ISS Python Course/day01/test_bitcoin_search_result.png')

    driver.implicitly_wait(5) # Let the user actually see something!

    assert 'No results found.' not in driver.page_source

    driver.close()
    
    return cypto_curr_df

In [14]:
bitcoin_crypto_df = getCryptoCurrMktCap('Bitcoin')

           Date       Market_Cap
0  Aug 04, 2018  127,859,350,021
1  Aug 03, 2018  129,965,898,111
2  Aug 02, 2018  131,189,721,712
3  Aug 01, 2018  133,492,645,169
4  Jul 31, 2018  140,559,547,538
5  Jul 30, 2018  141,239,444,979
6  Jul 29, 2018  140,951,371,776
7  Jul 28, 2018  140,305,137,664
8  Jul 27, 2018  136,532,811,776
9  Jul 26, 2018  140,404,998,144


In [4]:
ethereum_crypto_df = getCryptoCurrMktCap('Ethereum')

           Date      Market_Cap
0  Aug 01, 2018  43,839,225,862
1  Jul 31, 2018  46,191,809,198
2  Jul 30, 2018  47,150,243,495
3  Jul 29, 2018  47,149,572,096
4  Jul 28, 2018  47,418,966,016
5  Jul 27, 2018  46,837,104,640
6  Jul 26, 2018  47,667,482,624
7  Jul 25, 2018  48,421,990,400
8  Jul 24, 2018  45,509,849,088
9  Jul 23, 2018  46,338,043,904


In [6]:
#prepare the dataset for Bitcoin
bitcoin_crypto_df = bitcoin_crypto_df.sort_values(by = 'Date', ascending=True)
bitcoin_crypto_df['Date'] = pd.to_datetime(bitcoin_crypto_df['Date'], infer_datetime_format=True)
print (bitcoin_crypto_df.info())

#prepare the dataset for Ethereum
ethereum_crypto_df = ethereum_crypto_df.sort_values(by = 'Date', ascending=True)
ethereum_crypto_df['Date'] = pd.to_datetime(ethereum_crypto_df['Date'], infer_datetime_format=True)
print (ethereum_crypto_df.info())

bitcoin_dataset = go.Scatter(
    x=bitcoin_crypto_df['Date'],
    y=bitcoin_crypto_df['Market_Cap'],
    name='Bitcoin Market Cap (FYTD) data',
    line=dict(color='rgb(205, 12, 24)'),
    mode='lines+markers'
)
ethereum_dataset = go.Scatter(
    x=ethereum_crypto_df['Date'],
    y=ethereum_crypto_df['Market_Cap'],
    name='Ethereum Market Cap (FYTD) data',
    line=dict(color='rgba(115,115,115,1)'),
    mode='lines+markers',
    yaxis='y2'
)
data = [bitcoin_dataset, ethereum_dataset]
layout = go.Layout(
    title='Bitcoin v.s. Ethereum Mkt Cap - (FYTD Figures)',
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(82, 82, 82)',
        linewidth=1,
        autotick=True,
        ticks='outside',
        tickcolor='rgb(82, 82, 82)',
        tickwidth=2,
        ticklen=5,
        nticks=20,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        title='S$',
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgba(115,115,115,1)',
        linewidth=1,
        autotick=True
    ),
    yaxis2=dict(
        title='Mkt Cap in Billions',
        titlefont=dict(
            color='rgb(205, 12, 24)'
        ),
        tickfont=dict(
            color='rgb(204, 204, 204)'
        ),
        overlaying='y',
        side='right'
    ),
    legend=dict(
    orientation='h',
    y=1.1
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='multiple-axes-double')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119 entries, 118 to 0
Data columns (total 2 columns):
Date          119 non-null datetime64[ns]
Market_Cap    119 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 2.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 119 entries, 118 to 0
Data columns (total 2 columns):
Date          119 non-null datetime64[ns]
Market_Cap    119 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 2.8+ KB
None


In [33]:
#incorporate unit testing into the function with py UnitTest Module
import unittest
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import datetime


class CryptoCurrMktCap(unittest.TestCase):
    """CryptoCurrMktCap python test case"""
    
     # set the values at the class level rather than at method level - minimize the resource usage
    @classmethod
    def setUpClass(cls):
        """ Setting up for the test """
        print ("CryptoCurrMktCap:setUp_:begin")
        
        cls.driver = webdriver.Chrome(r'C:/Users/leongyld/AppData/Local/Continuum/anaconda3/Lib/site-packages/selenium/webdriver/common/chromedriver.exe')
        # navigate to the cryptocurr home page
        cls.driver.get('https://coinmarketcap.com/')
        cls.driver.maximize_window()
        # allow the website to load all its ajax/javascripts in the bg
        cls.driver.set_page_load_timeout(20)

    # ending the tests - Will be called after ALL tests are completed
    @classmethod
    def tearDownClass(cls):
        """Cleaning up after the test"""
        print ("CryptoCurrMktCap:tearDown_:begin")
        # close the browser window
        cls.driver.quit()
                     
    # test routine to determine if the website homepage is successfully loaded
    def testWebsiteHomepageSuccess(self):
        """Test routine A"""
        self.assertEqual('Cryptocurrency Market Capitalizations | CoinMarketCap', self.driver.title)
        self.captureTestEvidence('C:/Users/leongyld/Desktop/NUS ISS Python Course/day01/testEvidence/',self.testWebsiteHomepageSuccess.__name__)
        #print ("CryptoCurrMktCap:testA")
     
    # test routine to determine whether the searchbox widget is available
    def testCryptoCurrSearchbox(self):
        """Test routine B"""
        #searchbox_elem = self.driver.find_element_by_xpath("//*[@class='form-control quick-search-box js-quick-search tt-input']")
        self.assertTrue(self.is_element_present(By.XPATH,"//*[@class='form-control quick-search-box js-quick-search tt-input']"))
        self.captureTestEvidence('C:/Users/leongyld/Desktop/NUS ISS Python Course/day01/testEvidence/',self.testCryptoCurrSearchbox.__name__)        
        #print ("CryptoCurrMktCap:testB")
        
    # test routine to determine search results are returned when I key in 'Bitcoin' from searchbox widget
    def testCryptoCurrSearchboxResults(self,crypto_curr='Bitcoin'):
        """Test routine C"""
        searchbox_elem = self.driver.find_element_by_xpath("//*[@class='form-control quick-search-box js-quick-search tt-input']")
        searchbox_elem.clear()
        searchbox_elem.send_keys(crypto_curr)
        self.driver.implicitly_wait(5) # Let the user actually see something!
        #traverse thru using xpath to locate the dropdownlist results
        searchboxresult_elem = self.driver.find_element_by_xpath("//*[@class='tt-suggestions']")
        self.assertTrue('{}'.format(crypto_curr) in searchboxresult_elem.text)
        self.captureTestEvidence('C:/Users/leongyld/Desktop/NUS ISS Python Course/day01/testEvidence/',self.testCryptoCurrSearchboxResults.__name__)        
        #print ("CryptoCurrMktCap:testC")
        
    # test routine to determine that Bitcoin page is available
    def testCryptoCurrPageIsLoaded(self,crypto_curr='Bitcoin'):
        """Test routine D"""
        searchbox_elem = self.driver.find_element_by_xpath("//*[@class='form-control quick-search-box js-quick-search tt-input']")
        searchbox_elem.clear()
        searchbox_elem.send_keys(crypto_curr)
        searchbox_elem.send_keys(Keys.RETURN)
        self.assertEqual(True, ("https://coinmarketcap.com/currencies/bitcoin/" in self.driver.current_url))
        self.captureTestEvidence('C:/Users/leongyld/Desktop/NUS ISS Python Course/day01/testEvidence/',self.testCryptoCurrPageIsLoaded.__name__)        
        #print ("CryptoCurrMktCap:testD")
        
    # test routine to determine that Bitcoin historical panel is available with data
    def testCryptoCurrHistoricalPanel(self,crypto_curr='Bitcoin'):
        """Test routine E"""
        crypto_hist_link = self.driver.find_element_by_xpath("//*[@href='/currencies/"+crypto_curr.lower()+"/historical-data/']")
        self.assertTrue(self.is_element_present(By.XPATH,"//*[@href='/currencies/"+crypto_curr.lower()+"/historical-data/']"))
        crypto_hist_link.send_keys(Keys.ENTER)
        
        historicaldata_elem=self.driver.find_element_by_id('historical-data')
        historicaldata_elem.location_once_scrolled_into_view #scroll to the particular elem
        
        self.driver.implicitly_wait(5) # Let the user actually see something!
        
        #retrieve the YTD performance of the crytocurrency
        reportType_ddl_elem = self.driver.find_element_by_id("reportrange")
        self.assertTrue(self.is_element_present(By.ID,"reportrange"))

        reportType_ddl_elem.click()
        
        reportType_item = reportType_ddl_elem.find_element_by_xpath("//li[@data-range-key='Year To Date']")
        self.assertTrue(self.is_element_present(By.XPATH,"//li[@data-range-key='Year To Date']"))
        
        reportType_item.click()
        
        #scroll down to ensure that FYTD timespan is selected
        historicaldata_elem=self.driver.find_element_by_id('historical-data')
        historicaldata_elem.location_once_scrolled_into_view #scroll to the particular elem
        
        #test that the latest record = SYSDATE
        tbl_rec_elem = self.driver.find_element_by_xpath("//*[@id='historical-data']/div/div[2]/table/tbody/tr[1]/td[1]")
        self.assertEqual(True, ((datetime.datetime.now() - datetime.timedelta(days=2)).strftime('%b %d, %Y') in tbl_rec_elem.text))
        #self.assertEqual(True, ((datetime.datetime.now()).strftime('%b %d, %Y') in tbl_rec_elem.text))
        self.driver.implicitly_wait(5) # Let the user actually see something!
        self.captureTestEvidence('C:/Users/leongyld/Desktop/NUS ISS Python Course/day01/testEvidence/',self.testCryptoCurrHistoricalPanel.__name__)
        #print ("CryptoCurrMktCap:testE")
        
    def is_element_present(self, how, what):
        """
        Helper method to confirm the presence of an element on page
        :params how: By locator type
        :params what: locator value
        """
        try: self.driver.find_element(by=how, value=what)
        except NoSuchElementException: return False
        return True    
    
    def captureTestEvidence(self,filepath,filename):
        #capture the screenshot as test evidence!
        self.driver.save_screenshot(r'{}{}.png'.format(filepath,filename))
        
    def suite_test_cases():
        suite = unittest.TestSuite()
        suite.addTest(CryptoCurrMktCap('testWebsiteHomepageSuccess'))
        suite.addTest(CryptoCurrMktCap('testCryptoCurrSearchbox'))
        suite.addTest(CryptoCurrMktCap('testCryptoCurrSearchboxResults'))
        suite.addTest(CryptoCurrMktCap('testCryptoCurrPageIsLoaded'))
        suite.addTest(CryptoCurrMktCap('testCryptoCurrHistoricalPanel'))
        return suite
               
# Run the test cases by preparing the test suite and trigger the testrunner run func
if __name__ == '__main__':
    #CryptoCurrMktCapSuite = unittest.TestLoader().loadTestsFromTestCase(CryptoCurrMktCap)
    testRunner = unittest.TextTestRunner(descriptions=True, verbosity=3)
    testRunner.run(CryptoCurrMktCap.suite_test_cases())

CryptoCurrMktCap:setUp_:begin


testWebsiteHomepageSuccess (__main__.CryptoCurrMktCap)
Test routine A ... ok
testCryptoCurrSearchbox (__main__.CryptoCurrMktCap)
Test routine B ... ok
testCryptoCurrSearchboxResults (__main__.CryptoCurrMktCap)
Test routine C ... ok
testCryptoCurrPageIsLoaded (__main__.CryptoCurrMktCap)
Test routine D ... ok
testCryptoCurrHistoricalPanel (__main__.CryptoCurrMktCap)
Test routine E ... ok


CryptoCurrMktCap:tearDown_:begin



----------------------------------------------------------------------
Ran 5 tests in 31.530s

OK
